Case Study: Refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python
if you leverage functions as first-class objects.

Example 10-1. Implementation of the Order class with pluggable discount
strategies.

In [2]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional, Callable

class Customer(NamedTuple):
    name: str
    fidelity: int

class LineItem(NamedTuple):
    product: str
    quantity: Decimal
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity

class Order(NamedTuple):
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'

class Promotion(ABC):
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""
    
class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate 
        return Decimal(0)
    
class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [32]:
joe = Customer('John Doe', 0)
joe

Customer(name='John Doe', fidelity=0)

In [33]:

ann = Customer('Ann Smith', 1100)
ann

Customer(name='Ann Smith', fidelity=1100)

In [34]:

cart = (LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))

In [35]:

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [36]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

Example 10-1 works perfectly well, but the same functionality can be
implemented with less code in Python by using functions as objects. The
next section shows how.

Example 10-3. Order class with discount strategies implemented as
functions.

In [3]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Order:
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due:{self.due():.2f}>'

def fidelity_promo(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity
        points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount

def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

    

Example 10-7 is a somewhat hackish way of using globals to help
best_promo automatically find the other available *_promo functions.

Example 10-7. The promos list is built by introspection of the module
global namespace

In [4]:
promos = [promo for name, promo in globals().items()
            if name.endswith('_promo') and 
            name != 'best_promo']

def best_promo(order: Order) -> Decimal:
    return max(promo(order) for promo in promos)
    

In [10]:
joe = Customer('John Doe', 0)
joe
ann = Customer('Ann Smith', 1100)
ann
cart = (LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5)))
Order(joe, cart, best_promo)

<Order total: 42.00 due:42.00>

In Example 10-8, the only significant change is that the list of strategy
functions is built by introspection of a separate module called
promotions. Note that Example 10-8 depends on importing the
promotions module as well as inspect, which provides high-level
introspection functions.

Example 10-8. The promos list is built by introspection of a new promotions
module

In [12]:
import inspect

promos = [func for _, func in inspect.getmembers(Promotion, inspect.isfunction)]

def best_promo(order: Order) -> Decimal:
    return max(promo(order) for promo in promos) 

Decorator-Enhanced Strategy Pattern
Recall that our main issue with Example 10-6 is the repetition of the
function names in their definitions and then in the promos list used by the
best_promo function to determine the highest discount applicable. The
repetition is problematic because someone may add a new promotional
strategy function and forget to manually add it to the promos list—in
which case, best_promo will silently ignore the new strategy,
introducing a subtle bug in the system. Example 10-9 solves this problem
with the technique covered in “Registration decorators”.
Example 10-9. The promos list is filled by the promotion decorator

In [5]:
Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []

def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo

def best_promo(order: Order) -> Decimal:
    "return compute the best discount available"
    return max(promo(order) for promo in promos)

@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customer with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount

@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)
